In [1]:
# 소켓을 사용하기 위해서는 socket을 import해야 한다.
import socket, threading
import numpy as np
import pandas as pd
import re
import shutil
import os
import unicodedata
import urllib3
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

filename = 'kor-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)  
lines = pd.read_csv('kor.txt', names=['en','ko'], sep='\t', index_col=False)  

def returnTranslateText(message):
      lines.en[0]= message
      lines.en[0]
      lines.ko[0]="인"
      lines.ko[0]  
      lines.ko = lines.ko.apply(lambda x: '\t '+x+' \n')  
      # en & ko 글자 집합 구축
      en_vocab = set()
      for line in lines.en :
        for char in line:
          en_vocab.add(char)

      ko_vocab = set()
      for line in lines.ko :
        for char in line:
          ko_vocab.add(char)
        
      en_vocab_size = len(en_vocab)+1
      ko_vocab_size = len(ko_vocab)+1
        
      en_vocab = sorted(list(en_vocab))    
      ko_vocab = sorted(list(ko_vocab))  
        
      en_idx = dict([(word, i+1) for i, word in enumerate(en_vocab)])
      ko_idx = dict([(word, i+1) for i, word in enumerate(ko_vocab)])  
      
      # 입력 구성
      encoder_input = []
      for line in lines.en :
        temp_X = []
        for w in line:
          temp_X.append(en_idx[w])   #char - int 변환
        encoder_input.append(temp_X)
      
      # 출력 구성
      decoder_input = []
      for line in lines.ko :
        temp_X = []
        for w in line:
          temp_X.append(ko_idx[w])   #char - int 변환
        decoder_input.append(temp_X)
      # ko column 의 \t 제거
      decoder_target = []
      for line in lines.ko :
        t=0
        temp_X = []
        for w in line:
          if t>0:
            temp_X.append(ko_idx[w])
          t=t+1
        decoder_target.append(temp_X)
    
      # max length
      max_en_len = max([len(line) for line in lines.en])
      max_ko_len = max([len(line) for line in lines.ko])
        
      # 최대 길이를 10으로 나누어 사용 (학습 시간 단축)\n",
      max_en_len //= 10
      max_ko_len //= 10

      encoder_input = pad_sequences(encoder_input, maxlen=max_en_len, padding='post')
      decoder_input = pad_sequences(decoder_input, maxlen=max_ko_len, padding='post')
      decoder_target = pad_sequences(decoder_target, maxlen=max_ko_len, padding='post')
        
      # 원핫 벡터
      encoder_input = to_categorical(encoder_input)
      decoder_input = to_categorical(decoder_input)
      decoder_target = to_categorical(decoder_target)
      
      encoder_inputs = Input(shape=(None, en_vocab_size))
      encoder_lstm = LSTM(units=2048, return_state=True)
      encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
      encoder_states = [state_h, state_c] #은닉 상태, 셀 상태
      
      decoder_inputs = Input(shape=(None, ko_vocab_size)) 
      decoder_lstm = LSTM(units=2048, return_sequences=True, return_state=True)
      decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)

      decoder_softmax_layer = Dense(ko_vocab_size, activation='softmax')
      decoder_outputs = decoder_softmax_layer(decoder_outputs)

      # embedding 계층 관련 학습!
      model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
      model.compile(optimizer="adam", loss="categorical_crossentropy")

      model.load_weights('./kopago_model.hdf5')
      encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)
        
      # 이전 시점의 상태들을 저장하는 텐서
      decoder_state_input_h = Input(shape=(2048,))
      decoder_state_input_c = Input(shape=(2048,))
      decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
      decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
      # 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용. 이는 뒤의 함수 decode_sequence()에 구현
      decoder_states = [state_h, state_c]
      # 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
      decoder_outputs = decoder_softmax_layer(decoder_outputs)
      decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)   
      idx_to_en = dict((i, char) for char, i in en_idx.items())
      idx_to_ko = dict((i, char) for char, i in ko_idx.items())
      seq_index = 0
      input_seq = encoder_input[0:1]
        
      states_value = encoder_model.predict(input_seq)

      # <SOS>에 해당하는 원-핫 벡터 생성
      target_seq = np.zeros((1, 1, ko_vocab_size))
      target_seq[0, 0, ko_idx['\t']] = 1.

      stop_condition = False
      decoded_sentence = ""

      # stop_condition이 True가 될 때까지 루프 반복
      while not stop_condition:
          # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
          output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

          # 예측 결과를 문자로 변환
          sampled_token_index = np.argmax(output_tokens[0, -1, :])
          sampled_char = idx_to_ko[sampled_token_index]

          # 현재 시점의 예측 문자를 예측 문장에 추가
          decoded_sentence += sampled_char

          # <eos>에 도달하거나 최대 길이를 넘으면 중단.
          if (sampled_char == '\n' or
             len(decoded_sentence) > max_ko_len):
             stop_condition = True

          # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
          target_seq = np.zeros((1, 1, ko_vocab_size))
          target_seq[0, 0, sampled_token_index] = 1.

          # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
          states_value = [h, c]
     
      print(decoded_sentence)
   
      return decoded_sentence[:len(decoded_sentence)-1]

In [ ]:
# binder함수는 서버에서 accept가 되면 생성되는 socket 인스턴스를 통해 client로 부터 데이터를 받으면 echo형태로 재송신하는 메소드이다.
def binder(client_socket, addr):
  # 커넥션이 되면 접속 주소가 나온다.
  print('Connected by', addr)
  try:
    # 접속 상태에서는 클라이언트로 부터 받을 데이터를 무한 대기한다.
    # 만약 접속이 끊기게 된다면 except가 발생해서 접속이 끊기게 된다.
    while True:
      # socket의 recv함수는 연결된 소켓으로부터 데이터를 받을 대기하는 함수입니다. 최초 4바이트를 대기합니다.
      data = client_socket.recv(4)
      # 최초 4바이트는 전송할 데이터의 크기이다. 그 크기는 little big 엔디언으로 byte에서 int형식으로 변환한다.
      # C#의 BitConverter는 big엔디언으로 처리된다.
      length = int.from_bytes(data, "little")
      # 다시 데이터를 수신한다.
      data = client_socket.recv(length)
      # 수신된 데이터를 str형식으로 decode한다.
      msg = data.decode()
      # 수신된 메시지를 콘솔에 출력한다.
      print('Received from', addr, msg)
      #========================================================================================================
      msg = returnTranslateText(msg)
      #msg = "번역글에 대한 답장입니다!"
      #========================================================================================================
      # 수신된 메시지 앞에 「echo:」 라는 메시지를 붙힌다.
      msg = msg
      # 바이너리(byte)형식으로 변환한다.
      data = msg.encode()
      # 바이너리의 데이터 사이즈를 구한다.
      length = len(data)
      # 데이터 사이즈를 little 엔디언 형식으로 byte로 변환한 다음 전송한다.
      client_socket.sendall(length.to_bytes(4, byteorder='little'))
      # 데이터를 클라이언트로 전송한다.
      client_socket.sendall(data)
  except:
    # 접속이 끊기면 except가 발생한다.
    print("except : " , addr)
  finally:
    # 접속이 끊기면 socket 리소스를 닫는다.
    client_socket.close()
 
# 소켓을 만든다.
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# 소켓 레벨과 데이터 형태를 설정한다.
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
# 서버는 복수 ip를 사용하는 pc의 경우는 ip를 지정하고 그렇지 않으면 None이 아닌 ''로 설정한다.
# 포트는 pc내에서 비어있는 포트를 사용한다. cmd에서 netstat -an | find "LISTEN"으로 확인할 수 있다.
server_socket.bind(('', 9998))
# server 설정이 완료되면 listen를 시작한다.
server_socket.listen()
 
try:
  # 서버는 여러 클라이언트를 상대하기 때문에 무한 루프를 사용한다.
  while True:
    # client로 접속이 발생하면 accept가 발생한다.
    # 그럼 client 소켓과 addr(주소)를 튜플로 받는다.
    client_socket, addr = server_socket.accept()
    th = threading.Thread(target=binder, args = (client_socket,addr))
    # 쓰레드를 이용해서 client 접속 대기를 만들고 다시 accept로 넘어가서 다른 client를 대기한다.
    th.start();
except:
  print("server")
finally:
   # 에러가 발생하면 서버 소켓을 닫는다.
  server_socket.close()

Connected by ('127.0.0.1', 52917)
Received from ('127.0.0.1', 52917) I hope Tom is coming.
 톰이 오고 있길 바라. 

Received from ('127.0.0.1', 52917) 
 이거 좀 빌려 줄래? 

except :  ('127.0.0.1', 52917)
